In [1]:
!pip install spotipy
!pip install wheel
!pip install pandas
!pip install psycopg2

You should consider upgrading via the 'C:\Users\Usuario\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Usuario\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Usuario\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\Usuario\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [43]:
with open("C:/Users/Usuario/Documents/client_secret_spotify.txt",'r') as f:
    pwd= f.read()

In [44]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
client_id = '65a8d201a0fb46a095fa066b70f08123'
client_secret = pwd
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [58]:
# Creando la conexión a Redsshift
import psycopg2
url="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com"
data_base="data-engineer-database"
user="glo_tarcaya_coderhouse"
with open("C:/Users/Usuario/Documents/pwd_redshift.txt",'r') as f:
    pwd= f.read()
try:
    conn = psycopg2.connect(
        host='data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com',
        dbname=data_base,
        user=user,
        password=pwd,
        port='5439'
    )
    print("Conectado a Redshift con éxito!")
    
except Exception as e:
    print("No es posible conectar a Redshift")
    print(e)

Conectado a Redshift con éxito!


In [59]:
#Crear la tabla si no existe
with conn.cursor() as cur:
    cur.execute("""
        CREATE TABLE IF NOT EXISTS glo_tarcaya_coderhouse.canciones
        (
	    id VARCHAR(50) primary key  
	    ,artista VARCHAR(255)   
	    ,cancion VARCHAR(255)  
	    ,genero VARCHAR(300)   
	    ,album VARCHAR(100)   
	    ,total_canciones_album INTEGER  
	    ,reproducciones INTEGER 
	    , anio INTEGER   
	    ,duracion_ms INTEGER   
	    ,album_img VARCHAR(300) 
        )
    """)
    conn.commit()

In [50]:
#Vaciar la tabla para evitar duplicados o inconsistencias
cur.execute("Truncate table canciones")
count = cur.rowcount

In [63]:
#consultando la tabla
cur = conn.cursor()
cur.execute("SELECT * FROM canciones")
results = cur.fetchall()
results

[('6AQbmUe0Qwf5PZnt4HmTXv',
  'PinkPantheress',
  'Boys a liar Pt. 2',
  '',
  'Boys a liar Pt. 2',
  2,
  98,
  2023,
  131013,
  'https://i.scdn.co/image/ab67616d0000b27342c5ba689b2e7cbc208a8fa7'),
 ('3dnP0JxCgygwQH9Gm7q7nb',
  'Eslabon Armado',
  'Ella Baila Sola',
  'corrido, corridos tumbados, nueva musica mexicana, sad sierreno, sierreno',
  'Ella Baila Sola',
  1,
  99,
  2023,
  165671,
  'https://i.scdn.co/image/ab67616d0000b2732071a0c79802d9375a53bfef'),
 ('59uQI0PADDKeE6UZDTJEe8',
  'Morgan Wallen',
  'Last Night',
  'contemporary country',
  '3 Songs At A Time Sampler',
  3,
  91,
  2023,
  163854,
  'https://i.scdn.co/image/ab67616d0000b273fc1df8423733f6f3c9e8dea2'),
 ('0yLdNVWF3Srea0uzk55zFn',
  'Miley Cyrus',
  'Flowers',
  'pop',
  'Flowers',
  1,
  100,
  2023,
  200454,
  'https://i.scdn.co/image/ab67616d0000b273f429549123dbe8552764ba1d'),
 ('7K3BhSpAxZBznislvUMVtn',
  'Morgan Wallen',
  'Last Night',
  'contemporary country',
  'One Thing At A Time',
  36,
  89,
  20

In [65]:
#Consultando los datos
import pandas as pd
results = sp.search(q='year:2023', type='track', limit=50)
data = {'Id': [],'Artista': [], 'Cancion': [],'Duracion_ms': [], 'Genero': [],'Album': [], 'Album_img': [], 'Total_canciones_album': [], 'Reproducciones': [], 'Anio': []}
for track in results['tracks']['items']:
    id = track['id']
    artist_name = track['artists'][0]['name']
    artist_id = track['artists'][0]['id']
    track_name = track['name']
    duration_ms = track['duration_ms']
    track_id = track['id']
    album_group = track['album']['name']
    album_img = track['album']['images'][0]['url'] #imagen de album
    album_cont = track['album']['total_tracks']
    track_genre = sp.artist(artist_id)['genres']
    track_popularity = track['popularity']
    track_year = track['album']['release_date'][:4]
    
    track_name = track_name.replace("'", "")
    album_group = album_group.replace("'", "")
    track_genre = ', '.join(track_genre)

    data['Id'].append(id)
    data['Artista'].append(artist_name)
    data['Cancion'].append(track_name)
    data['Duracion_ms'].append(duration_ms)
    data['Album'].append(album_group)
    data['Album_img'].append(album_img)
    data['Total_canciones_album'].append(album_cont)
    data['Genero'].append(track_genre)
    data['Reproducciones'].append(track_popularity)
    data['Anio'].append(track_year)
df = pd.DataFrame(data)
display(df)

KeyboardInterrupt: 

In [61]:
#Insertando los datos en Redsfhift
from psycopg2.extras import execute_values
with conn.cursor() as cur:
    execute_values(
        cur,
        '''
        INSERT INTO canciones (Id, Artista, Cancion, Duracion_ms, Genero, Album, Album_img, Total_canciones_album,Reproducciones,Anio)
        VALUES %s
        ''',
        [tuple(row) for row in df.values],
        page_size=len(df)
    )
    conn.commit()

In [55]:
# from psycopg2.extras import execute_values
# cur = conn.cursor()

# for track in results['tracks']['items']:
#     id = track['id']
#     artist_name = track['artists'][0]['name']
#     artist_id = track['artists'][0]['id']
#     track_name = track['name']
#     duration_ms = track['duration_ms']
#     track_id = track['id']
#     album_group = track['album']['name']
#     album_img = track['album']['images'][0]['url'] 
#     album_cont = track['album']['total_tracks']
#     track_genre = sp.artist(artist_id)['genres']
#     track_popularity = track['popularity']
#     track_year = track['album']['release_date'][:4]
    
#     track_name = track_name.replace("'", "")
#     album_group = album_group.replace("'", "")
#     track_genre = ', '.join(track_genre)
#     cur.execute(f"INSERT INTO canciones (Id, Artista, Cancion, Duracion_ms, Genero, Album, Album_img, Total_canciones_album,Reproducciones,Anio) VALUES ('{id}','{artist_name}', '{track_name}', {duration_ms}, '{track_genre}', '{ album_group}','{album_img}', {album_cont}, {track_popularity}, {track_year})")
#     conn.commit()


In [64]:
cur.close()
conn.close()